In [ ]:
from pyspark import SparkContext
import string as s

In [ ]:
sc = SparkContext("local","TEST")

In [ ]:
file=sc.textFile("test2.csv")
file.collect()

In [ ]:
file.getNumPartitions()

In [ ]:
file.cache()
#file.persist()
file.getStorageLevel().useMemory

In [ ]:
file.is_cached

In [ ]:
file.unpersist()

In [ ]:
file.is_cached

In [ ]:
file.getStorageLevel().useMemory

In [ ]:
file.map(lambda x: (x.split(",")[0],x.split(",")[3])).collect()

In [ ]:
flat=file.flatMap(lambda x: (x.split(",")[1],x.split(",")[2]))
type(flat)
len(flat.collect())

In [ ]:

#Adding conditions for rdd using filter,
file2=sc.textFile("test3.csv")

#removing the first line from file as it is header
first_line=file2.first()
rdd2=file2.filter(lambda x: x != first_line)     #remove header
rdd3=rdd2.filter(lambda x: x != "")              #remove last empty line
#rdd3.collect()

#rdd3.map(lambda x: int(x.split(",")[2])).collect()
rdd3.filter(lambda x: int(x.split(",")[2]) > 5000).collect() #gives you enitre x if conditions matches

#rdd3.map(lambda x: x[0],filter(lambda y: int(y.split(",")[2]) > 5000, rdd3)).collect() # does not work
rdd3.map(lambda x:x.split(",")[0] if (int(x.split(",")[2]) > 5000) else "").collect() #doesnot work without else


In [ ]:
rdd = sc.parallelize(range(100), 4)
#rdd.collect()
rdd.sample(True, 0.05, 80).collect()  #[11, 33, 33, 42, 74]  33 is repeated becuase of True(withReplacement)
rdd.sample(False, 0.05, 80).collect()  #[16, 17, 61, 72, 88] 80 is the starting number(seed)

In [ ]:
file.intersection(rdd4).collect() #common elements

In [ ]:
file.union(file2).collect() #add rdd

In [ ]:
file2.distinct().collect()

In [ ]:
rdd3.collect()

In [ ]:
rdd3.sortBy(lambda x: int(x.split(",")[2]),False).collect()

In [ ]:
rdd4=rdd3.repartition(3)
rdd4.collect()
rdd4.getNumPartitions()

In [ ]:
#********You have to use lambda function on all operations except mapPartitions, as they iterate themselves

In [ ]:
def fun(partition):
        for element in partition:
                yield element.split(",")[2]
rdd4.mapPartitions(fun).collect()

In [ ]:
def fun(index,partition):
        salary=[]
        for element in partition:
            salary.append(element.split(",")[2])
        yield (index,salary)
rdd4.mapPartitionsWithIndex(fun).collect()

In [ ]:
list1=rdd4.groupBy(lambda x: x.split(",")[0]).collect()  #returns pyspark.resultiterable.ResultIterable since...
                                                    # ..we are grouping by name, we get two columns of data

#lets see what is there in first element
tuple1=list1[0]
for x in (tuple1[1]):
    print(x)
    
    
#else you can also do a map on groupby, so that you directly get a list
rdd4.groupBy(lambda x: x.split(",")[0]).map(lambda x: (x[0], list(x[1]))).collect()

In [ ]:
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2)])
r=sorted(x.cogroup(y).collect())              #cogroup groups to rdds,no need to have any 
print (r)
for i in r:
     for j in i:
            if (type(j) == 'str'):
                print (j)
            else:
                for k in j:
                    print (list(k))

In [ ]:
rdd4.keyBy(lambda x: int(x.split(",")[2])*2).collect() #same as map but with key value pairs

In [ ]:
#another complex example of keyby and cogroup together
x = sc.parallelize(range(0,3)).keyBy(lambda x: x*x)
y = sc.parallelize(zip(range(0,5), range(0,5)))
[(x, list(map(list, y))) for x, y in sorted(x.cogroup(y).collect())]

In [ ]:
rdd8=file.repartition(3)         #repartition created equal size
rdd4.zip(rdd8).collect()

In [ ]:
rdd4.zipWithIndex().collect()

In [ ]:
#coalese will increase or decrease partitions.uses exsisting partition. less shuffle compared 
#to repartiton and hence fast. Hence unequal partitions.
rdd9=rdd4.coalesce(2)
rdd9.glom().collect()     #converts everything to list

In [76]:
sc.stop()